# Set name of computer  

In [ ]:
pc_name = "MacBook_M1_Pro"

# Standard benchmarking notebook

In [ ]:
import os
import yaml

import numpy as np
from nanopyx.core.transform import NLMDenoising, Convolution2D, CRShiftAndMagnify

def concatenate_benchmarks(results, args_repr, previous_dict):
    if args_repr not in previous_dict:
        previous_dict[args_repr] = {}
        
    for t2run,name,_ in results:
        if name in previous_dict[args_repr]:
            previous_dict[args_repr][name].append(t2run)
        else:
            previous_dict[args_repr][name] = [t2run]
    
    return previous_dict


denoising = NLMDenoising(verbose=False,testing=True)
denoising._run_types.pop('threaded_dynamic')
denoising._run_types.pop('threaded_static')
denoising._run_types.pop('threaded_guided')

conv = Convolution2D(verbose=False, testing=True)
conv._run_types.pop('threaded_dynamic')
conv._run_types.pop('threaded_static')
conv._run_types.pop('threaded_guided')

crsm = CRShiftAndMagnify(verbose=False, testing=True)
crsm._run_types.pop('threaded_dynamic')
crsm._run_types.pop('threaded_static')
crsm._run_types.pop('threaded_guided')

denoising_results = {}
conv_results = {}
crsm_results = {}

n_repeats = 2

## 2D Convolution

In [ ]:
conv_image_1 = np.random.random((500, 500)).astype(np.float32)
conv_image_2 = np.random.random((2500, 2500)).astype(np.float32)

kernel_1 = np.ones((5,5)).astype(np.float32)
kernel_2 = np.ones((9,9)).astype(np.float32)
kernel_3 = np.ones((21,21)).astype(np.float32)

args_1 = [conv_image_1, kernel_1]
kwargs_1 = {}
args_repr_1,_ = conv._get_args_repr_score(*args_1, **kwargs_1)

args_2 = [conv_image_2, kernel_2]
kwargs_2 = {}
args_repr_2,_ = conv._get_args_repr_score(*args_2, **kwargs_2)

args_3 = [conv_image_2, kernel_3]
kwargs_3 = {}
args_repr_3,_ = conv._get_args_repr_score(*args_3, **kwargs_3)

In [ ]:
for i in range(n_repeats): 
    results = conv.benchmark(*args_1, **kwargs_1)
    conv_results = concatenate_benchmarks(results, args_repr_1, conv_results)
    results = conv.benchmark(*args_2, **kwargs_2)
    conv_results = concatenate_benchmarks(results, args_repr_2, conv_results)
    results = conv.benchmark(*args_3, **kwargs_3)
    conv_results = concatenate_benchmarks(results, args_repr_3, conv_results)

## Non-local means denoising

In [ ]:
nlm_image_1 = np.random.random((1, 1000, 1000)).astype(np.float32)
nlm_image_2 = np.random.random((1, 500, 500)).astype(np.float32)
nlm_image_3 = np.random.random((1, 200, 200)).astype(np.float32)

args_1 = [nlm_image_1]
kwargs_1 = {'patch_size': 50, 'patch_distance': 50, 'h': 0.1, 'sigma': 0.0}
args_repr_1,_ = conv._get_args_repr_score(*args_1, **kwargs_1)

args_2 = [nlm_image_2]
kwargs_2 = {'patch_size': 5, 'patch_distance': 100, 'h': 0.1, 'sigma': 0.0}
args_repr_2,_ = conv._get_args_repr_score(*args_2, **kwargs_2)

args_3 = [nlm_image_3]
kwargs_3 = {'patch_size': 5, 'patch_distance': 10, 'h': 0.1, 'sigma': 0.0}
args_repr_3,_ = conv._get_args_repr_score(*args_3, **kwargs_3)

In [ ]:
for i in range(n_repeats): 
    results = denoising.benchmark(*args_1, **kwargs_1)
    denoising_results = concatenate_benchmarks(results, args_repr_1, denoising_results)
    results = denoising.benchmark(*args_2, **kwargs_2)
    denoising_results = concatenate_benchmarks(results, args_repr_2, denoising_results)
    results = denoising.benchmark(*args_3, **kwargs_3)
    denoising_results = concatenate_benchmarks(results, args_repr_3, denoising_results)


## Catmul-Rom Shift and Magnify

In [ ]:
crsm_image_1 = np.random.random((10, 10, 10)).astype(np.float32)
crsm_image_2 = np.random.random((10, 100, 100)).astype(np.float32)
crsm_image_3 = np.random.random((100, 300, 300)).astype(np.float32)

args_1 = [crsm_image_1,0, 0, 5, 5]
kwargs_1 = {}
args_repr_1,_ = conv._get_args_repr_score(*args_1, **kwargs_1)

args_2 = [crsm_image_2, 0, 0, 5, 5]
kwargs_2 = {}
args_repr_2,_ = conv._get_args_repr_score(*args_2, **kwargs_2)

args_3 = [crsm_image_3, 0, 0, 5, 5]
kwargs_3 = {}
args_repr_3,_ = conv._get_args_repr_score(*args_3, **kwargs_3)

In [ ]:
for i in range(n_repeats): 
    results = crsm.benchmark(*args_1, **kwargs_1)
    crsm_results = concatenate_benchmarks(results, args_repr_1, crsm_results)
    results = crsm.benchmark(*args_2, **kwargs_2)
    crsm_results = concatenate_benchmarks(results, args_repr_2, crsm_results)
    results = crsm.benchmark(*args_3, **kwargs_3)
    crsm_results = concatenate_benchmarks(results, args_repr_3, crsm_results)

In [ ]:
if not os.path.exists("Benchmark_results"):
    os.mkdir("Benchmark_results")

with open(f"Benchmark_results/{pc_name}_Results_denoising.yaml", 'w') as f:
    yaml.dump(denoising_results, f)

with open(f"Benchmark_results/{pc_name}_Results_conv.yaml", 'w') as f:
    yaml.dump(conv_results, f)

with open(f"Benchmark_results/{pc_name}_Results_crsm.yaml", 'w') as f:
    yaml.dump(crsm_results, f)